In [1]:
import pandas as pd

splits = {'train': 'Sentiment_train.csv', 'validation': 'Sentiment_valid .csv', 'test': 'Sentiment_test.csv'}
data = pd.read_csv("hf://datasets/mounikaiiith/Telugu_Sentiment/" + splits["train"])

In [2]:
data.head()

,Unnamed: 0,Sentence,Sentiment
0,33982,అయితే ఆయన దర్శక త్వం వహించరు కానీ ఆయన స్క్రిప్...,neutral
1,20617,వీరిద్దరూ మంచి మిత్రులు కావడంతో తరచూ ఫోన్‌లో మ...,neutral
2,27596,బాలేక‌పోతే మాత్రం ముందే చెప్పానుగా ఇలా జ‌రుగుత...,neg
3,15772,అమిత్ షా ర్యాలీని అడ్డుకోవ‌డానికి జ‌రిగిన ఘ‌ర్...,neg
4,33930,రిక్వె స్ట్‌ పేరుతో మీడియా ప్రతినిధులు ఎలా వ్య...,neutral


In [3]:
data.shape

(24599, 3)

In [4]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [18]:
data['Sentiment'][24595]

'neutral'

In [6]:
data['Sentiment'].value_counts()

Sentiment
neutral    13213
pos         6635
neg         4751
Name: count, dtype: int64

In [7]:
import pandas as pd
from sklearn.utils import resample

sentiment_counts = data['Sentiment'].value_counts()

# Separate by sentiment labels
neutral_data = data[data['Sentiment'] == 'neutral']
pos_data = data[data['Sentiment'] == 'pos']
neg_data = data[data['Sentiment'] == 'neg']

# Undersample the majority class (neutral) to match the size of the minority classes
neutral_undersampled = resample(neutral_data, replace=False, n_samples=min(sentiment_counts['pos'], sentiment_counts['neg']), random_state=42)

# Combine the undersampled majority class with the minority classes
balanced_data = pd.concat([neutral_undersampled, pos_data, neg_data])


balanced_data['Sentiment'].value_counts()

Sentiment
pos        6635
neutral    4751
neg        4751
Name: count, dtype: int64

In [9]:
import re
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel


# Function to preprocess Telugu text
def preprocess_text(text):
    # Remove special characters, numbers, and punctuations (retain Telugu characters and spaces)
    text = re.sub(r'[^\u0C00-\u0C7F\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply preprocessing
balanced_data['Cleaned_Sentence'] = balanced_data['Sentence'].apply(preprocess_text)

In [10]:
balanced_data.head()

,Sentence,Sentiment,Cleaned_Sentence
15588,అతడు ఆస్ట్రేలియా వెళ్లనున్నాడు’ అని తెలిపాడు.\n,neutral,అతడు ఆస్ట్రేలియా వెళ్లనున్నాడు అని తెలిపాడు
8896,"వైకుంఠం క్యూకాంప్లెక్స్‌లోని వంట‌శాల‌ను, వంట‌క...",neutral,వైకుంఠం క్యూకాంప్లెక్స్లోని వంటశాలను వంటకు విన...
24243,మూడు రోజులుగా జరుగుతున్న ప్రాపర్టీ అండ్ బిల్డ...,neutral,మూడు రోజులుగా జరుగుతున్న ప్రాపర్టీ అండ్ బిల్డ్...
2505,ఎవరిది పైచేయి..?.,neutral,ఎవరిది పైచేయి
8264,రాష్ట్రవ్యాప్తంగా మహాశుద్ధి కార్యక్రమాలను అధిక...,neutral,రాష్ట్రవ్యాప్తంగా మహాశుద్ధి కార్యక్రమాలను అధిక...


In [11]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
import pandas as pd

# Ensure model is on GPU
model = model.to("cuda")

# Function to get embeddings in batches
def get_embeddings_batch(sentences):
    # Tokenize the sentences and convert to tensors (batch processing)
    tokens = tokenizer(sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")
    
    # Move tokens to GPU
    tokens = {key: value.to("cuda") for key, value in tokens.items()}
    
    # Ensure no gradients are calculated
    with torch.no_grad():
        outputs = model(**tokens)
    
    # Use the [CLS] token embeddings for all sentences in the batch
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move embeddings to CPU
    return embeddings

# Split data into batches (e.g., 32 sentences per batch)
batch_size = 32
batches = [balanced_data['Cleaned_Sentence'][i:i + batch_size].tolist() for i in range(0, len(balanced_data), batch_size)]

# Process each batch and store embeddings
embeddings_list = []
for batch in batches:
    embeddings = get_embeddings_batch(batch)
    embeddings_list.append(embeddings)

# Flatten the list of embeddings and add to DataFrame
embeddings_array = np.vstack(embeddings_list)
balanced_data['Embeddings'] = list(embeddings_array)

# Display the dataset with embeddings
print(balanced_data[['Cleaned_Sentence', 'Embeddings']])


                                        Cleaned_Sentence  \
15588        అతడు ఆస్ట్రేలియా వెళ్లనున్నాడు అని తెలిపాడు   
8896   వైకుంఠం క్యూకాంప్లెక్స్లోని వంటశాలను వంటకు విన...   
24243  మూడు రోజులుగా జరుగుతున్న ప్రాపర్టీ అండ్ బిల్డ్...   
2505                                       ఎవరిది పైచేయి   
8264   రాష్ట్రవ్యాప్తంగా మహాశుద్ధి కార్యక్రమాలను అధిక...   
...                                                  ...   
24586              డిపార్ట్మెంట్ ని కూడా వార్న్ చేస్తారు   
24588  ఇటీవల ఈ చిత్ర పోస్టర్ విడుదల కాగా ఇందులో అంజలి...   
24589  ఉదయం దీక్ష కొనసాగిస్తున్న భట్టి విక్రమార్కను అ...   
24591       మాకు భారతీయ సినిమాల గురించి ఎక్కువగా తెలియదు   
24592  టీ సిరీస్ అనంతరం భారత్తో నాలుగు మ్యాచ్ల టెస్టు...   

                                              Embeddings  
15588  [0.09581549, -0.29468456, -0.28114238, 0.19447...  
8896   [-0.023697907, -0.056628086, -0.31600136, 0.10...  
24243  [-0.028214905, -0.035800166, -0.27313882, 0.00...  
2505   [-0.03465656, 0.039982524, -0.022354

In [13]:
balanced_data.to_csv('balanced_data_with_embeddings.csv', index=False)


In [14]:
balanced_data.head()

,Sentence,Sentiment,Cleaned_Sentence,Embeddings
15588,అతడు ఆస్ట్రేలియా వెళ్లనున్నాడు’ అని తెలిపాడు.\n,neutral,అతడు ఆస్ట్రేలియా వెళ్లనున్నాడు అని తెలిపాడు,"[0.09581549, -0.29468456, -0.28114238, 0.19447..."
8896,"వైకుంఠం క్యూకాంప్లెక్స్‌లోని వంట‌శాల‌ను, వంట‌క...",neutral,వైకుంఠం క్యూకాంప్లెక్స్లోని వంటశాలను వంటకు విన...,"[-0.023697907, -0.056628086, -0.31600136, 0.10..."
24243,మూడు రోజులుగా జరుగుతున్న ప్రాపర్టీ అండ్ బిల్డ...,neutral,మూడు రోజులుగా జరుగుతున్న ప్రాపర్టీ అండ్ బిల్డ్...,"[-0.028214905, -0.035800166, -0.27313882, 0.00..."
2505,ఎవరిది పైచేయి..?.,neutral,ఎవరిది పైచేయి,"[-0.03465656, 0.039982524, -0.022354579, 0.270..."
8264,రాష్ట్రవ్యాప్తంగా మహాశుద్ధి కార్యక్రమాలను అధిక...,neutral,రాష్ట్రవ్యాప్తంగా మహాశుద్ధి కార్యక్రమాలను అధిక...,"[-0.0854372, -0.17006357, -0.24240623, 0.06859..."


In [15]:
balanced_data.shape

(16137, 4)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Convert embeddings into a format suitable for model training (e.g., numpy array)
X = np.array(balanced_data['Embeddings'].tolist())

# Labels (Sentiment)
y = balanced_data['Sentiment']

# Apply label encoding to the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Verify the shape of the split data
print(f"Training data size: {X_train.shape}, Testing data size: {X_test.shape}")
print(f"Encoded Sentiments: {label_encoder.classes_}")



Training data size: (12909, 768), Testing data size: (3228, 768)
Encoded Sentiments: ['neg' 'neutral' 'pos']


In [17]:
y_train

array([1, 0, 2, ..., 1, 0, 2])

In [26]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Initialize the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Define a simplified parameter grid for XGBoost
xgb_param_grid = {
    'learning_rate': [0.01, 0.05],        # Reduce the number of learning rates
    'n_estimators': [100],                 # Keep only one value for n_estimators
    'max_depth': [3, 6],                   # Limit the range for max_depth
    'subsample': [0.8],                    # Keep a single value for subsample
    'colsample_bytree': [0.8]              # Keep a single value for colsample_bytree
}

# Perform GridSearchCV with the simplified hyperparameters
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_xgb_model = xgb_grid_search.best_estimator_
print(f"Best XGBoost Model: {best_xgb_model}")



Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best XGBoost Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)


In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Initialize the RandomForest model
rf_model = RandomForestClassifier(random_state=42)

# Define a simplified parameter grid for RandomForest
rf_param_grid = {
    'n_estimators': [100, 200],              # Fewer values for number of trees
    'max_depth': [10, 20],                    # Limited max depth
    'min_samples_split': [2],                  # Only one option for min samples to split
    'min_samples_leaf': [1],                   # Only one option for min samples at leaf
    'bootstrap': [True]                        # Keep bootstrap sampling as True
}

# Perform GridSearchCV with the simplified hyperparameters
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
rf_grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_rf_model = rf_grid_search.best_estimator_
print(f"Best RandomForest Model: {best_rf_model}")



Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best RandomForest Model: RandomForestClassifier(max_depth=20, n_estimators=200, random_state=42)


In [29]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate XGBoost model
y_pred_xgb = best_xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Model Accuracy: {xgb_accuracy}")
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

# Evaluate RandomForest model
y_pred_rf = best_rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"RandomForest Model Accuracy: {rf_accuracy}")
print("RandomForest Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Select the best model based on accuracy
if xgb_accuracy > rf_accuracy:
    best_model = best_xgb_model
    print("XGBoost is the best model.")
else:
    best_model = best_rf_model
    print("RandomForest is the best model.")


XGBoost Model Accuracy: 0.5427509293680297
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.56      0.56       918
           1       0.50      0.26      0.34       952
           2       0.54      0.73      0.62      1358

    accuracy                           0.54      3228
   macro avg       0.54      0.52      0.51      3228
weighted avg       0.54      0.54      0.52      3228

RandomForest Model Accuracy: 0.5154894671623296
RandomForest Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.47      0.52       918
           1       0.47      0.17      0.25       952
           2       0.50      0.79      0.61      1358

    accuracy                           0.52      3228
   macro avg       0.51      0.48      0.46      3228
weighted avg       0.51      0.52      0.48      3228

XGBoost is the best model.


In [30]:
import joblib

# Save the best model to a pickle file
joblib.dump(best_model, 'best_model.pkl')
print("Best model saved to 'best_model.pkl'.")


Best model saved to 'best_model.pkl'.
